In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bitsandbytes # 모델을 경량화해서 로드
!pip install transformers
!pip install peft # LoRA 튜닝을 하게 해주는 라이브러리
!pip install accelerate
!pip install trl
!pip install datasets
!pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [3]:
from transformers import AutoTokenizer, LlamaForCausalLM, StoppingCriteria, StoppingCriteriaList
import torch
import argparse
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import re
import gc
import warnings
warnings.filterwarnings("ignore")

In [4]:
# 한번만 하면 됨
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# test_df = pd.read_csv('../test.csv')
test_df = pd.read_csv('/content/drive/MyDrive/데이콘/test.csv')
def remove_quotes(text):
    text = re.sub(r'"', '', text)
    return text
test_df['질문'] = test_df['질문'].apply(remove_quotes)
test_df

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,압출법 보온판의 가장 큰 장점은 무엇인가요?
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [6]:
model_id = "beomi/llama-2-koen-13b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, eos_token="<|endoftext|>")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}) # 모델을 쪼개서 load함
model = prepare_model_for_kbit_training(model)
# Adapter의 폴더 명을 가져와야 함
# adapter_name = "llama2_13b_finetuned_preprocess_data_aug/checkpoint-7000"
adapter_name = "/content/drive/MyDrive/데이콘/llama-2-koen-13b/llama2_13b_finetuned_preprocess_data_aug"
# model에 아까 학습한 adapter를 연결시킴
model = PeftModel.from_pretrained(model, adapter_name)

tokenizer_config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/2.96G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/474M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [7]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(46336, 5120, padding_idx=2)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_la

In [8]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [9]:
tokenizer

LlamaTokenizerFast(name_or_path='beomi/llama-2-koen-13b', vocab_size=46336, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46331: AddedToken("<|sep|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46332: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46333: AddedToken("<|acc|>", rstrip=False, lstrip=False, single_word=False, normalized=False, spe

In [10]:
class LocalStoppingCriteria(StoppingCriteria):

    def __init__(self, tokenizer, stop_words=[]):
        super().__init__()

        stops = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for
                 stop_word in stop_words]
        print('stop_words', stop_words)
        print('stop_words_ids', stops)
        self.stop_words = stop_words
        self.stops = [stop.cuda() for stop in stops]
        self.tokenizer = tokenizer

    def _compare_token(self, input_ids):
        for stop in self.stops:
            if len(stop.size()) != 1:
                continue
            stop_len = len(stop)
            if torch.all((stop == input_ids[0][-stop_len:])).item():
                return True

        return False

    def _compare_decode(self, input_ids):
        input_str = self.tokenizer.decode(input_ids[0])
        for stop_word in self.stop_words:
            if input_str.endswith(stop_word):
                return True
        return False

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        input_str = self.tokenizer.decode(input_ids[0])
        for stop_word in self.stop_words:
            if input_str.endswith(stop_word):
                return True
        return False

In [11]:
stop_words = ["<|endoftext|>", "###", "</s>"]
stopping_criteria = StoppingCriteriaList([LocalStoppingCriteria(tokenizer=tokenizer, stop_words=stop_words)])

stop_words ['<|endoftext|>', '###', '</s>']
stop_words_ids [tensor(46332), tensor(835), tensor(2)]


In [12]:
pipe = pipeline(task = "text-generation",
        model=model,
        tokenizer=tokenizer,
        do_sample=True,
        temperature=0.01,
        num_beams=3,
        top_p=0.99,
        top_k=3,
        epsilon_cutoff=9e-4,
        eta_cutoff=2e-3,
        penalty_alpha=1.5,
        max_length=1000,
        # max_new_tokens=1024,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2,
        # no_repeat_ngram_size=30,
        stopping_criteria=stopping_criteria,
        framework='pt')

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [13]:
def ask(x):
    ans = pipe(
        f"당신은 질문에 답변하는 역할을 하는 챗봇입니다. 사용자의 질문에 올바른 답변을 하세요.\n### 질문: {x}\n### 답변: "
    )
    answer = ans[0]['generated_text'].split('### 답변: ')[1].strip()
    del ans
    torch.cuda.empty_cache()
    return answer

In [ ]:
test_df.loc[0, "질문"]

'방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?'

In [ ]:
answer = ask(test_df.loc[0, "질문"])
print(answer)

낡은 목재 가구, 창틀, 문틀 등을 리모델링 할 때에는 방청 페인트를 사용해야 합니다. 방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다. 또한, 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 단점으로 꼽힙니다. 이러한 단점들을 고려하여 건물의 외벽 재질을 선택할 때에는 신중한 판단과 관리가 필요하다는 점을 염두에 두어야 합니다.


In [ ]:
test_df.loc[1, "질문"]

'도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?'

In [ ]:
answer = ask(test_df.loc[1, "질문"])
print(answer)

낡은 도배지에서 녹이 발생하는 것은 일반적으로 철재 부속품인 못, 자재 등을 사용하여 발생합니다. 이러한 구성 요소들이 녹이 나면 도배지 표면에 녹이 묻어나올 수 있습니다. 따라서 도배지 녹 방지를 위해 부속품들의 재질과 제조사를 주의깊게 고려해야 합니다. 또한, 도배지 시공 전에는 녹 방지 페인트를 사용하여 도배지 표면에 녹이 나는 것을 방지할 수 있습니다. 만약 도배지 녹이 이미 발생하였다면, 각 성분들을 분리하여 녹을 제거한 후 재시공을 해야 합니다. 녹 방지 페인트를 사용하여 도배지 표면을 처리하면 추가적인 녹을 방지할 수 있습니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다. 따라서, 도배지 녹 방지를 위해서는 도배지 소재와 구성 요소를 고려하여 적절한 대책을 취하는 것이 중요합니다. 만약 도배지 녹이 발생한다면, 전문가의 도움을 받아 재시공하는 것이 좋습니다. 녹 방지 페인트를 사용하여 추가적인 방지를 할 수도 있습니다. 따라서, 도배지 녹 방지를 위해 적절한 대책을 취하는 것이 중요합니다. 만약 도배지 녹이 발생한다면, 전문가의 도움을 받아 재시공하는 것이 좋습니다. 녹 방지 페인트를 사용하여 추가적인 방지를 할 수도 있습니다. 따라서, 도배지 녹 방지를 위해 적절한 대책을 취하는 것이 중요합니다. 만약 도배지 녹이 발생한다면, 전문가의 도움을 받아 재시공하는 것이 좋습니다. 녹 방지 페인트를 사용하여 추가적인 방지를 할 수도 있습니다. 따라서, 도배지 녹 방지를 위해 적절한 대책을 취하는 것이 중요합니다. 만약 도배지 녹이 발생한다면, 전문가의 도움을 받아 재시공하는 것이 좋습니다. 녹 방지 페인트를 사용하여 추가적인 방지를 할 수도 있습니다. 따라서, 도배지 녹 방지를 위해 적절한 대책을 취하는 것이 중요합니다. 만약 도배지 녹이 발생한다면, 전문가의 도움을 받아 재시공하는 것이 좋습니다. 녹 방지 페인트를 사용하여 추가적인 방지를 할 수도 있습니다. 따라서, 도배지 녹 방지를 위해 적절한 대책을 취하는 것이 중요합니다. 만약 도배지 녹

In [ ]:
test_df.loc[3, "질문"]

'철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?'

In [ ]:
answer = ask(test_df.loc[3, "질문"])
print(answer)

2000년 이후에 건축된 철골구조 건물은 단열성능이 우수하며, 이는 고층 건물에서도 안정적으로 온도를 유지하는 데 도움을 줍니다. 따라서 고층 건물의 단열 효과를 높이기 위해서는 최신식의 철골구조를 사용하여, 열 전달을 최소화하는 것이 좋습니다. 또한, 단열재와 같은 부재를 설치하여 열 흐름을 차단하는 것도 효과적입니다. 이렇게 함으로써 고층 건물의 단열 효과를 높일 수 있습니다. 만약 새로운 철골구조를 건축하는 것이 어려운 상황이라면, 기존 구조물의 단열성을 향상시키는 방법도 고려할 만 합니다. 예를 들어, 단열재를 보강하거나 외부에서의 열 전달을 차단하는 방법 등을 통해 기존 구조물의 단열성을 향상시킬 수 있습니다. 따라서 고층 건물의 단열 효과를 높이기 위해서는 다양한 방법을 고려하여 적절한 대책을 마련하는 것이 중요합니다. 또한, 건물의 용도, 위치, 주변 환경 등을 고려하여 적절한 대책을 마련하는 것이 중요합니다. 만약 이러한 사항들이 고려되지 않으면, 고층 건물의 단열 효과가 기대에 미치지 못할 수 있습니다. 따라서 이러한 사항들을 고려하여 적절한 대책을 마련하는 것이 중요합니다. 마지막으로, 고층 건물의 단열 효과를 높이기 위해서는 전문가의 도움을 받는 것이 좋습니다. 건축가 또는 관련 전문가와 상의하여 적절한 대책을 마련하는 것이 좋습니다. 만약 이러한 전문가의 도움을 받지 않으면, 고층 건물의 단열 효과가 기대에 미치지 못할 수 있습니다. 따라서 고층 건물의 단열 효과를 높이기 위해서는 관련 전문가의 도움을 받는 것이 중요합니다. 이를 통해 고층 건물의 단열 효과를 높일 수 있습니다.
###


In [ ]:
test_df.loc[5, "질문"]

'철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?'

In [ ]:
answer = ask(test_df.loc[5, "질문"])
print(answer)

2000년대 이후에 주로 사용된 철근철골콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건축 구조입니다. 이 구조는 철골의 강도와 내구성을 바탕으로 건물을 지지하고, 철근은 인장력을, 콘크리트는 압축력을 담당하여 건물의 강성과 튼튼함을 보장합니다. 이러한 구조는 대규모의 건물이나 고층 건물에서 안정성을 확보하는 데 적합하며, 화재에도 상대적으로 안전합니다. 또한 내화성과 내진성능이 뛰어나지만, 시공과정이 복잡하고 공사기간이 길며, 균일한 시공이 어렵다는 단점이 있습니다. 이러한 이유로 철근철골콘크리트는 철골과 철근콘크리트의 장점을 결합하여 안정성을 높이고 건물을 지탱하는 구조로 평가됩니다. 다만, 철근철골콘크리트 구조에서도 철근의 인장력이 상대적으로 약점으로 지적되고 있습니다. 이러한 약점을 보완하기 위해 최근에는 철근의 인장력을 향상시키는 연구가 진행되고 있습니다. 또한, 철근철골콘크리트 구조는 건축물의 형태를 자유롭게 연출할 수 있는 장점을 가지고 있습니다. 다만, 이러한 구조의 안정성이 충분히 확보되지 않을 경우 건물의 형태를 자유롭게 연출하기 어렵다는 단점이 있습니다. 이러한 이유로 철근철골콘크리트 구조는 안정성과 형태자유를 고려하여 건물을 설계하고 건설하는 것이 중요합니다. 따라서 건축물의 안정성과 형태를 고려하여 적합한 구조를 선택하는 것이 필요합니다. 이러한 이유로 철근철골콘크리트 구조는 안정성과 형태자유를 고려하여 건물을 설계하고 건설하는 것이 중요합니다. 따라서 건축물의 안정성과 형태를 고려하여 적합한 구조를 선택하는 것이 필요합니다. 이러한 구조는 철골의 강도와 내구성을 바탕으로 건물을 지지하고, 철근은 인장력을, 콘크리트는 압축력을 담당하여 건물의 강성과 튼튼함을 보장합니다. 이러한 구조는 대규모의 건물이나 고층 건물에서 안정성을 확보하는 데 적합하며, 화재에도 상대적으로 안전합니다. 또한 내화성과 내진성능이 뛰어나지만, 시공과정이 복잡하고 공사기간이 길며, 균일한 시공이 어렵다는 단점이 있습니다. 이러한 이유로 철

In [ ]:
test_df.loc[19, "질문"]

'다이닝 룸을 더 고급스럽게 꾸밀려면 어떻게 해야 하나요와 아레카 야자를 키우면 어떤 이점이 있는지의 관계는 무엇인가요?'

In [ ]:
answer = ask(test_df.loc[19, "질문"])
print(answer)

풋라이트 및 간접 조명을 활용하여 다이닝 룸을 분위기 있게 꾸밀 수 있습니다. 또한, 아레카 야자는 NASA에서 선정한 공기정화식물 1위에 해당하는 식물로 포름알데히드, 트리클로, 에틸렌, 벤젠 등의 실내 독성물질VOC를 효과적으로 제거하는 것으로 알려져 있습니다. 또한, 아레카 야자의 잎은 하루에 1리터의 수분을 공기 중으로 방출해서 실내 습도를 높이는 효과가 있다고 합니다. 이와 함께 아름다운 외모와 함께 공기정화 및 습도 조절 기능을 제공하여 건강한 실내 환경을 조성하는 데 도움을 줍니다. 물론, 관리를 잘 해야 하며, 적절한 조명과 습도 관리가 필요합니다. 하지만 이를 통해 건강한 실내 환경을 조성하는 데 도움을 줄 수 있습니다. 또한, 아레카 야자는 주로 실내에서 키우기 때문에 실내 환경에 적합하며, 공기를 정화하고 습도를 조절하는 데 도움이 됩니다. 또한, 아레카 야자의 잎은 하루에 1리터의 수분을 공기 중으로 방출해서 실내 습도를 높이는 효과가 있다고 합니다. 이와 함께 아름다운 외모와 함께 공기정화 및 습도 조절 기능을 제공하여 건강한 실내 환경을 조성하는 데 도움을 줍니다. 물론, 관리를 잘 해야 하며, 적절한 조명과 습도 관리가 필요합니다. 하지만 이를 통해 건강한 실내 환경을 조성하는 데 도움을 줄 수 있습니다. 또한, 아레카 야자는 주로 실내에서 키우기 때문에 실내 환경에 적합하며, 공기를 정화하고 습도를 조절하는 데 도움이 됩니다. 또한, 아레카 야자의 잎은 하루에 1리터의 수분을 공기 중으로 방출해서 실내 습도를 높이는 효과가 있다고 합니다. 이와 함께 아름다운 외모와 함께 공기정화 및 습도 조절 기능을 제공하여 건강한 실내 환경을 조성하는 데 도움을 줍니다. 물론, 관리를 잘 해야 하며, 적절한 조명과 습도 관리가 필요합니다. 하지만 이를 통해 건강한 실내 환경을 조성하는 데 도움을 줄 수 있습니다. 또한, 아레카 야자는 주로 실내에서 키우기 때문에 실내 환경에 적합하며, 공기를 정화하고 습도를 조절하는 데 도움이 됩니다. 또한, 아

In [ ]:
test_df.loc[16, "질문"]

'준불연재료는 무엇인가요? 그리고 유성페인트를 사용하는 것에 대한 부작용이 있을까요?'

In [ ]:
answer = ask(test_df.loc[16, "질문"])
print(answer)

2007년 건축법 개정으로 도입된 준불연재료는 10분간 화재에 견딜 수 있는 소재를 의미합니다. 또한, 유성페인트는 주로 석유나 석탄에서 추출한 기름이나 화학물질로 만들어지며, 이들 중 일부는 인체에 유해한 영향을 미칠 수 있습니다. 유성페인트의 주요 단점은 다음과 같습니다. 1. 유해성: 유성페인트에는 벤젠, 톨루엔, 자일렌 등의 유해물질이 포함되어 있으며, 이러한 물질들은 호흡기나 피부 등을 통해 체내에 흡수될 수 있습니다. 장기간 노출되면 암, 호흡기 질환, 피부 질환 등의 건강 문제를 유발할 수 있습니다. 2. 냄새: 유성페인트는 강한 냄새를 가지고 있으며, 이는 작업자뿐만 아니라 주변 사람들에게도 불쾌감을 줄 수 있습니다. 3. 건조 시간: 유성페인트는 건조 시간이 길며, 이로 인해 작업 시간이 더 오래 걸릴 수 있습니다. 또한, 건조 과정에서 발생하는 열로 인해 작업자가 불편함을 느낄 수 있습니다. 4. 환경 오염: 유성페인트는 환경 오염을 유발할 수 있습니다. 유성페인트의 성분 중 일부는 대기 중에 방출되면 미세먼지나 오존 등의 오염물질을 생성할 수 있습니다. 5. 유지보수 어려움: 유성페인트는 유지보수가 어렵습니다. 유성페인트는 물에 녹지 않기 때문에 청소나 보수 작업 시 추가적인 도구와 방법이 필요합니다. 6. 비용: 유성페인트는 일반적으로 수성페인트보다 가격이 비쌉니다. 또한, 유성페인트는 환경에 더 많은 오염을 유발하므로, 친환경적인 선택이 아닙니다. 이러한 이유로, 유성페인트를 사용할 때에는 안전하고 적절한 환경에서 사용하고, 꼭 필요한 경우에만 사용하는 것이 좋습니다. 또한, 유성페인트를 사용할 때에는 마스크와 같은 개인보호구를 착용하고, 환기를 충분히 시켜 물과 공기 중의 유해한 물질의 농도를 낮추는 것이 중요합니다. 유성페인트를 처리할 때에는 폐기규칙을 준수하여 안전하게 처리하는 것이 필요합니다. 환경부와 식품의약품안전처에서는 유성페인트에 대한 유해성 정보와 안전한 취급, 폐기에 대한 지침을 제공하고 있습니다. 이를 통해 유성페인트를

In [ ]:
test_df.loc[62, "질문"]

'인테리어 디자인에서 에나멜 계열 페인트를 사용하여 공간의 색상을 선택할 때 고려해야 할 요소들은 무엇인가요?'

In [ ]:
answer = ask(test_df.loc[62, "질문"])
print(answer)

2023년의 최신 트렌드로는 천연 소재와 어스톤 컬러 팔레트가 여전히 유행할 것으로 예상됩니다. 또한, 친환경적이고 지속 가능한 디자인이 더욱 중요시되며, 실내 공간에서 식물을 활용한 자연친화적 디자인이 강조될 것으로 보입니다. 또한, 디지털 기술과 전통적 요소를 결합한 현대적이면서도 따뜻한 느낌의 복합 디자인이 주목받을 것으로 예상됩니다. 이러한 트렌드를 고려하여 2023년의 인테리어 디자인을 설계하는 것이 중요합니다. 특히 천연 소재와 어스톤 컬러 팔레트는 꾸준히 인기가 있을 것으로 예상되므로 이러한 요소들을 고려하여 디자인을 수립하는 것이 좋습니다. 또한, 디지털 기술과 전통적 요소를 결합한 현대적이면서도 따뜻한 느낌의 복합 디자인이 주목받을 것으로 예상되므로 이러한 트렌드를 반영하여 공간을 꾸미는 것이 중요합니다. 따라서 2023년의 최신 트렌드를 고려하여 인테리어 디자인을 수립하는 것이 중요합니다. 특히 디지털 기술과 전통적 요소를 결합한 현대적이면서도 따뜻한 느낌의 복합 디자인이 주목받을 것으로 예상되므로 이러한 트렌드를 반영하여 공간을 꾸미는 것이 중요합니다. 또한, 실내 공간에서 식물을 활용한 자연친화적 디자인이 강조될 것으로 예상되므로 식물을 활용하여 자연스러운 분위기를 조성하는 것이 좋습니다. 이러한 최신 트렌드를 고려하여 2023년의 인테리어 디자인을 설계하는 것이 중요합니다. 특히 디지털 기술과 전통적 요소를 결합한 현대적이면서도 따뜻한 느낌의 복합 디자인이 주목받을 것으로 예상되므로 이러한 트렌드를 반영하여 공간을 꾸미는 것이 중요합니다. 또한, 실내 공간에서 식물을 활용한 자연친화적 디자인이 강조될 것으로 예상되므로 식물을 활용하여 자연스러운 분위기를 조성하는 것이 좋습니다. 이러한 최신 트렌드를 고려하여 2023년의 인테리어 디자인을 설계하는 것이 중요합니다. 특히 천연 소재와 어스톤 컬러 팔레트가 꾸준히 인기가 있을 것으로 예상되므로 이러한 요소들을 고려하여 디자인을 수립하는 것이 좋습니다. 또한, 디지털 기술과 전통적 요소를 결합

### model inference

In [14]:
preds = []
for t in tqdm(test_df['질문'], total = len(test_df)):
    result = ask(t)
    if not re.findall(r"\d+\.", result):
        result = '.'.join(result.split('.')[:9]) + '.'
        result = result.replace('..', '.')
    find = result.find('#')
    if find != -1:
        result = result[:find]
    find = result.find('</s>')
    if find != -1:
        result = result[:find]
    # Remove URLs
    result = re.sub(r'https?://\S+|www\.\S+', '', result)
    result = result.strip()
    print('------------------------')
    print('질문: ', t)
    print('------------------------')
    print('답변: ', result)
    preds.append(result)

  0%|          | 0/130 [00:00<?, ?it/s]

------------------------
질문:  방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
------------------------
답변:  낡은 목재 가구, 창틀, 문틀 등을 리모델링 할 때에는 방청 페인트를 사용해야 합니다. 방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다. 또한, 원목사이딩은 색상이 변색될 수 있고 해충 침입에 취약하다는 점도 단점으로 꼽힙니다. 이러한 단점들을 고려하여 건물의 외벽 재질을 선택할 때에는 신중한 판단과 관리가 필요하다는 점을 염두에 두어야 합니다.
------------------------
질문:  도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
------------------------
답변:  낡은 도배지에서 녹이 발생하는 것은 일반적으로 철재 부속품인 못, 자재 등을 사용하여 발생합니다. 이러한 구성 요소들이 녹이 나면 도배지 표면에 녹이 묻어나올 수 있습니다. 따라서 도배지 녹 방지를 위해 부속품들의 재질과 제조사를 주의깊게 고려해야 합니다. 또한, 도배지 시공 전에는 녹 방지 페인트를 사용하여 도배지 표면에 녹이 나는 것을 방지할 수 있습니다. 만약 도배지 녹이 이미 발생하였다면, 각 성분들을 분리하여 녹을 제거한 후 재시공을 해야 합니다. 녹 방지 페인트를 사용하여 도배지 표면을 처리하면 추가적인 녹을 방지할 수 있습니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다. 따라서, 도배지 녹 방지를 위해서는 도배지 소재와 구성 요소를 고려하여 적절한 대책을 취하는 것이 중요합니다. 만약 도배지 녹이 발생한다면, 전문가의 도움을 받아 재시공하는 것이 좋습니다.
-----------------

In [15]:
pd.DataFrame({'질문':test_df['질문'],'답변':preds}).to_csv('/content/drive/MyDrive/데이콘/llama-2-koen-13b/llama_2_13b_preprocess_data_aug_epoch6(text).csv', index = False)

### submission

In [16]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

(130, 512)

In [18]:
submit = pd.read_csv('/content/drive/MyDrive/데이콘/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.056289,0.019185,0.025814,-0.002345,0.050613,0.024002,-0.028298,-0.005344,0.017398,...,-0.004966,-0.079026,0.024386,-0.025508,0.007354,0.011021,0.059132,-0.002045,0.037806,0.044192
1,TEST_001,0.005273,0.050755,0.035070,0.002087,0.072573,-0.003941,-0.010209,-0.026707,-0.030663,...,-0.009801,-0.034938,0.041117,-0.017270,-0.016073,0.008931,0.015167,-0.073646,0.006330,0.010120
2,TEST_002,0.000861,-0.087621,-0.026650,0.019379,0.102170,-0.020446,0.018932,0.004222,0.016654,...,-0.032508,-0.031393,0.043686,-0.044781,0.001459,0.029236,-0.023376,-0.047361,0.009868,0.019772
3,TEST_003,0.019170,-0.034055,-0.035779,0.031909,0.057888,-0.006488,-0.063370,0.031863,-0.010380,...,-0.030850,-0.008803,0.031067,-0.052380,0.016303,0.003415,-0.023486,-0.042757,0.013549,0.026611
4,TEST_004,-0.002360,-0.033546,0.000603,-0.013551,0.092303,-0.033635,0.056511,0.032630,-0.026504,...,0.010824,-0.048844,0.025842,-0.004710,-0.036827,0.018267,-0.003275,0.017354,-0.002051,0.078064


In [19]:
submit.to_csv('/content/drive/MyDrive/데이콘/llama-2-koen-13b/llama_2_13b_preprocess_data_aug_epoch6.csv', index = False)
pd.read_csv('/content/drive/MyDrive/데이콘/llama-2-koen-13b/llama_2_13b_preprocess_data_aug_epoch6.csv')

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.056289,0.019185,0.025814,-0.002345,0.050613,0.024002,-0.028298,-0.005344,0.017398,...,-0.004966,-0.079026,0.024386,-0.025508,0.007354,0.011021,0.059132,-0.002045,0.037806,0.044192
1,TEST_001,0.005273,0.050755,0.035070,0.002087,0.072573,-0.003941,-0.010209,-0.026707,-0.030663,...,-0.009801,-0.034938,0.041117,-0.017270,-0.016073,0.008931,0.015167,-0.073646,0.006330,0.010120
2,TEST_002,0.000861,-0.087621,-0.026650,0.019379,0.102170,-0.020446,0.018932,0.004222,0.016654,...,-0.032508,-0.031393,0.043686,-0.044781,0.001459,0.029236,-0.023376,-0.047361,0.009868,0.019772
3,TEST_003,0.019170,-0.034055,-0.035779,0.031909,0.057888,-0.006488,-0.063370,0.031863,-0.010380,...,-0.030850,-0.008803,0.031067,-0.052380,0.016303,0.003415,-0.023486,-0.042757,0.013549,0.026611
4,TEST_004,-0.002360,-0.033546,0.000603,-0.013551,0.092303,-0.033635,0.056511,0.032630,-0.026504,...,0.010824,-0.048844,0.025842,-0.004710,-0.036827,0.018267,-0.003275,0.017354,-0.002051,0.078064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.029718,-0.051835,0.010206,0.015956,0.095478,-0.077506,0.056313,0.053033,0.062077,...,0.034476,0.011324,0.058722,-0.048884,0.020918,0.028653,0.038925,-0.045941,-0.029730,0.013741
126,TEST_126,0.002819,-0.055028,-0.083792,0.007844,0.072474,-0.021874,0.032018,0.028907,0.050656,...,-0.021755,-0.030953,0.037646,-0.024750,0.023831,0.035732,0.003923,-0.001995,-0.011363,0.044818
127,TEST_127,-0.033218,-0.005870,-0.012667,0.021730,0.082563,-0.021619,-0.005067,-0.036459,-0.018345,...,0.005824,-0.031373,0.005488,-0.043930,-0.017005,0.039709,0.026394,0.005982,-0.044413,0.026900
128,TEST_128,0.038883,-0.007835,-0.076268,0.010689,0.091179,-0.050459,0.031098,0.011224,-0.007407,...,-0.005116,0.033929,-0.003912,-0.037198,-0.005778,0.033262,-0.014792,0.000837,0.015204,0.009089
